In [1]:
from collections import OrderedDict
import pickle

from ml_pee_stego_v1 import MLPEEStego

# Set up the secret

In [2]:
from utils.data_preparation import get_filenames_from_folder, get_secret_file

# Tentukan folder path tempat secret
SECRET_FOLDER_PATH = 'secrets/bin'

# Dapatkan semua nama file .txt dari folder
secret_filenames = get_filenames_from_folder('txt', SECRET_FOLDER_PATH)
secret_filenames.sort(reverse=True)
print(secret_filenames)

# Masukkan ke dalam dictionary berisi nama filename beserta isi secret filenya
secret_dicts = OrderedDict((secret_filename, get_secret_file(
    SECRET_FOLDER_PATH + '/' + secret_filename + '.txt')) for secret_filename in secret_filenames)
# print(secret_dicts.get(secret_filenames[1]))

for secret_name, secret_data in secret_dicts.items():
    print(f"Secret untuk {secret_name}")

['secret_0.39_bps', 'secret_0.32_bps', 'secret_0.21_bps', 'secret_0.15_bps', 'secret_0.08_bps']
Secret untuk secret_0.39_bps
Secret untuk secret_0.32_bps
Secret untuk secret_0.21_bps
Secret untuk secret_0.15_bps
Secret untuk secret_0.08_bps


# Set up the model

In [3]:
import warnings
from sklearn.exceptions import InconsistentVersionWarning

# Menonaktifkan peringatan InconsistentVersionWarning
warnings.simplefilter("ignore", category=InconsistentVersionWarning)

models = OrderedDict()

models['SVR'] = 'models/svr_model.pkl'
models['SVR 2.0'] = 'models/svr_model_2.0.pkl'
models['LASSO'] = 'models/lasso_model.pkl'
models['ElasticNet'] = 'models/elastic_net_model.pkl'
models['Bayesian Ridge'] = 'models/bayesian_ridge_model.pkl'
models['SGD Regressor'] = 'models/sgd_model.pkl'

In [4]:
from utils.data_preparation import get_original_data

patient_code = '100'
original_data = get_original_data(patient_code=patient_code)

In [9]:
# Example
model_name = 'LASSO'
with open(models.get(model_name), 'rb') as model_file:
    model = pickle.load(model_file)

stego = MLPEEStego(model)
threshold = 4
for secret_name, secret_data in secret_dicts.items():
    print(f"Secret untuk {secret_name}")
    for key in ['001', '010', '011', '100', '101', '110', '111']:
        print(
            f"Model {model_name} secret {secret_name} threshold {threshold} key {key}")
        print(">>Embedding...")
        watermarked_data, result = stego.embed(
            original_data, secret_data, threshold=threshold, secret_key=key)
        print(">>Extracting...")
        extracted_original_data, extracted_secret_data = stego.extract(
            watermarked_data, threshold=threshold, secret_key=key)

        print(">>", original_data[0:20], secret_data[0:20])
        print(">>", extracted_original_data[0:20],
            extracted_secret_data[0:20])
        print(">>", "Len -> Ori:", len(original_data), ", Ex-ori:", len(extracted_original_data),
              ", Sec:", len(secret_data), ", Ex-sec:", len(extracted_secret_data), f", Rate ({key}): {len(extracted_secret_data)*100/(key.count('1')*1200)}%")
        print("******\n******")

Secret untuk secret_0.39_bps
Model LASSO secret secret_0.39_bps threshold 4 key 001
>>Embedding...
[-145 -145 -145 -145 -145 -145 -145 -148 -120 -135]
Total yang lebih dari/sama dengan threshold (T = 4): 694
Unhidden secret: 997
NCC: 0.9999477359050764
PRD: 0.4824676604549673
SNR: 46.330635837353206
Time: 0.3595571517944336
>>Extracting...
[-145 -145 -145 -145 -145 -145 -145 -145 -120 -135]
>> [-145 -145 -145 -145 -145 -145 -145 -145 -120 -135 -145 -150 -160 -155
 -160 -175 -180 -185 -170 -155] 11001001011110000011
>> [-145 -145 -145 -145 -145 -145 -145 -145 -120 -135 -145 -150 -160 -155
 -160 -175 -180 -185 -170 -155] 11001001011110000011
>> Len -> Ori: 3600 , Ex-ori: 3600 , Sec: 1404 , Ex-sec: 407 , Rate (001): 33.916666666666664%
******
******
Model LASSO secret secret_0.39_bps threshold 4 key 010
>>Embedding...
[-145 -145 -145 -145 -145 -145 -142 -145 -120 -138]
Total yang lebih dari/sama dengan threshold (T = 4): 654
Unhidden secret: 974
NCC: 0.9999450227236167
PRD: 0.497766648507

In [6]:
# Example
model_name = 'LASSO'
with open(models.get(model_name), 'rb') as model_file:
    model = pickle.load(model_file)

stego = MLPEEStego(model)
for secret_name, secret_data in secret_dicts.items():
    print(f"Secret untuk {secret_name}")
    for threshold in range(4, 10, 1):
        for key in ['001', '010', '011', '100', '101', '110', '111']:
            print(
                f"Model {model_name} secret {secret_name} threshold {threshold} key {key}")
            print(">>Embedding...")
            watermarked_data, result = stego.embed(
                original_data, secret_data, threshold=threshold, secret_key=key)
            print(">>Extracting...")
            extracted_original_data, extracted_secret_data = stego.extract(
                watermarked_data, threshold=threshold, secret_key=key)
            
            print(">>", original_data[0:20], secret_data[0:20])
            print(">>", extracted_original_data[0:20], extracted_secret_data[0:20])
            print(">>", "Len -> Ori:", len(original_data), ", Ex-ori:", len(extracted_original_data),
                  ", Sec:", len(secret_data), ", Ex-sec:", len(extracted_secret_data), f", Rate ({key}): {(key.count('1')*1200/len(extracted_secret_data))*100}%")
            print("******\n******")

Secret untuk secret_0.39_bps
Model LASSO secret secret_0.39_bps threshold 4 key 001
>>Embedding...
[-145 -145 -145 -145 -145 -145 -145 -148 -120 -135]
Total yang lebih dari/sama dengan threshold (T = 4): 694
Unhidden secret: 997
NCC: 0.9999477359050764
PRD: 0.4824676604549673
SNR: 46.330635837353206
Time: 0.3795442581176758
>>Extracting...
[-145 -145 -145 -145 -145 -145 -145 -145 -120 -135]
>> [-145 -145 -145 -145 -145 -145 -145 -145 -120 -135 -145 -150 -160 -155
 -160 -175 -180 -185 -170 -155] 11001001011110000011
>> [-145 -145 -145 -145 -145 -145 -145 -145 -120 -135 -145 -150 -160 -155
 -160 -175 -180 -185 -170 -155] 11001001011110000011
>> Len -> Ori: 3600 , Ex-ori: 3600 , Sec: 1404 , Ex-sec: 407 , Rate (001): 0.2457002457002457%
******
******
Model LASSO secret secret_0.39_bps threshold 4 key 010
>>Embedding...
[-145 -145 -145 -145 -145 -145 -142 -145 -120 -138]
Total yang lebih dari/sama dengan threshold (T = 4): 654
Unhidden secret: 974
NCC: 0.9999450227236167
PRD: 0.497766648507

KeyboardInterrupt: 

Cek alasan:
[x] Secret data berbeda
